In [ ]:
import pandas as pd
import numpy as np
from glob import glob
from datetime import datetime,timedelta
import pymongo
import os
import time

In [ ]:
def mongodb_index_data_save():
    try:
        today_date_str = datetime.today().strftime("%d%m%Y")
        input_file = f'Y:\\{today_date_str}\\index_data.txt'
        
        # Read and preprocess the data
        data = pd.read_csv(input_file, header=None, usecols=[0, 2, 3])
        data.rename(columns={0: 'index_name', 2: 'time_entry', 3: 'ltp'}, inplace=True)
        data['index_name'] = data['index_name'].str[23:-1]
        data['time_entry'] = data['time_entry'].str[16:-1]
        data['ltp'] = data['ltp'].str[8:-2]
        
        # Create MongoDB client and database
        database_name = f'index_data_{today_date_str}'
        client = pymongo.MongoClient("mongodb://localhost:27017/")
        db = client[database_name]
        
        # Group data by index_name and prepare for bulk insertion
        grouped_data = data.groupby('index_name')
        for index_name, group in grouped_data:
            group = group[['time_entry', 'ltp']]
            records = group.to_dict(orient='records')
            collection = db[index_name]
            collection.insert_many(records)
    except Exception as e:
        print(f'Error: {e}')

start_time = time.time()
if __name__ == '__main__':
    mongodb_index_data_save()
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Elapsed time: {elapsed_time:.2f} seconds")